In [3]:
import pandas as pd
import h5py

In [4]:
# Obtained from https://buzsakilab.nyumc.org/datasets/GrosmarkAD
file_name = 'Achilles_10252013_sessInfo.mat'
#file_name = 'Buddy_06272013_sessInfo.mat'
#file_name = 'Cicero_09012014_sessInfo.mat'
#file_name = 'Gatsby_08022013_sessInfo.mat'
f = h5py.File(file_name, 'r')
epoch = 'MazeEpoch'
t_start = f['sessInfo']['Epochs'][epoch][0][0]
t_stop = f['sessInfo']['Epochs'][epoch][1][0]
track_start = 0.2 # Only include data at least this far from the left-end of the track
track_stop = 1.4 # Only include data less than this far from the right-end of the track

In [5]:
# Load all the spike times and cell IDs
spikes = pd.DataFrame()
spikes['cell_ids'] = f['sessInfo']['Spikes']['SpikeIDs'][0].astype(int)
cell_ids = spikes['cell_ids'].unique()
spikes['t'] = f['sessInfo']['Spikes']['SpikeTimes'][0]
print("There are %d spikes across %d cells" % 
      (spikes.shape[0], len(spikes['cell_ids'].unique())))
in_epoch = (spikes['t']>t_start) & (spikes['t']<t_stop)
spikes = spikes[in_epoch]
print("In the epoch '%s', there are %d spikes across %d cells" % 
      (epoch, spikes.shape[0], len(cell_ids)))
spikes = spikes.set_index('t')

There are 8414607 spikes across 137 cells
In the epoch 'MazeEpoch', there are 865967 spikes across 137 cells


In [6]:
# Load all the location data
locations = pd.DataFrame()
locations['d'] = f['sessInfo']['Position']['OneDLocation'][0]
locations['x'] = f['sessInfo']['Position']['TwoDLocation'][0]
locations['y'] = f['sessInfo']['Position']['TwoDLocation'][1]
locations['t'] = f['sessInfo']['Position']['TimeStamps'][()]
in_epoch = (locations['t']>t_start) & (locations['t']<t_stop)
locations = locations[in_epoch].set_index('t').dropna()

In [7]:
# Join the spike and location data together, keeping only the times and locations of spikes
df = spikes.join(locations, how='outer').sort_index()
df.loc[:,['d','x','y']] = df.loc[:,['d','x','y']].interpolate(limit_area='inside')
on_track = (df['d']>track_start) & (df['d']<track_stop)
df = df[on_track]
on_track = (locations['d']>track_start) & (locations['d']<track_stop)
locations = locations[on_track]
df = df.dropna()
df['cell_ids'] = df['cell_ids'].astype(int)
cell_ids = df['cell_ids'].unique()
print("In the epoch '%s', there are %d spikes across %d cells that occurred during track running" % 
      (epoch, df.shape[0], len(cell_ids)))
df.head()

In the epoch 'MazeEpoch', there are 104048 spikes across 136 cells that occurred during track running


,cell_ids,d,x,y
t,,,,
18194.76930,609,0.200284,-0.061786,-0.189188
18194.77235,529,0.201205,-0.060900,-0.189143
18194.77305,1209,0.202125,-0.060015,-0.189098
18194.77855,416,0.203724,-0.058476,-0.189284
18194.77955,828,0.204402,-0.057822,-0.189515


In [9]:
df.to_csv('place_cell_spike_data.csv')
locations.to_csv('place_cell_location_data.csv')